# Research-Oriented Operating Systems: Chapter 4 - Inter-process Communication

This Jupyter Notebook explores **Inter-process Communication (IPC)** in operating systems, focusing on **Shared Memory** and **Message Passing**. It provides theoretical insights, practical simulations, visualizations, research directions, and projects for researchers and scientists.

## Objectives
- Understand shared memory and message passing mechanisms for IPC.
- Simulate IPC using Python's multiprocessing capabilities.
- Analyze performance and explore research trends.
- Develop hands-on skills through tutorials and projects.

## Prerequisites
- Python 3 with `multiprocessing`, `pandas`, `matplotlib`, `numpy`.
- VirtualBox and Ubuntu ISO for lab setup.
- Basic knowledge of OS concepts (*Operating System Concepts*, Chapter 3 or 4 recommended).

## Current Date and Time
- 11:30 AM IST, Wednesday, July 30, 2025


## 1. Shared Memory

### Theory
**Shared Memory** allows multiple processes to access a common memory region, enabling fast data exchange without copying.

- **Mechanism**: Processes map a shared memory segment into their address spaces.
- **Advantages**: High performance for large data transfers.
- **Challenges**: Requires synchronization (e.g., semaphores, locks) to prevent race conditions.

**Rare Insight**: Cache coherence in shared memory systems can degrade performance on multi-core CPUs due to frequent cache invalidation, a critical issue in high-performance computing.

**Applications**:
- **Databases**: Shared memory for in-memory data access (e.g., PostgreSQL).
- **Multimedia**: Fast data exchange in video processing pipelines.

### Practical Code: Shared Memory Simulation
Simulate two processes sharing a memory buffer using `multiprocessing.shared_memory`.


In [1]:
import multiprocessing as mp
import numpy as np
import time
import sys

try:
    from multiprocessing import shared_memory
except ImportError:
    print('multiprocessing.shared_memory is only available in Python 3.8+')
    sys.exit(1)

def producer(shm_name, size):
    shm = shared_memory.SharedMemory(name=shm_name)
    buffer = np.ndarray((size,), dtype=np.int32, buffer=shm.buf)
    for i in range(size):
        buffer[i] = i + 1  # Avoid zero to distinguish written values
        time.sleep(0.01)  # Simulate work
    shm.close()

def consumer(shm_name, size, result_queue):
    shm = shared_memory.SharedMemory(name=shm_name)
    buffer = np.ndarray((size,), dtype=np.int32, buffer=shm.buf)
    start_time = time.time()
    while any(buffer == 0):  # Wait for producer to fill all
        time.sleep(0.01)
    elapsed = time.time() - start_time
    result_queue.put((elapsed, buffer[:5].tolist()))
    shm.close()

# Simulate shared memory
if __name__ == '__main__':
    size = 10
    shm = shared_memory.SharedMemory(create=True, size=size * 4)
    buffer = np.ndarray((size,), dtype=np.int32, buffer=shm.buf)
    buffer[:] = 0

    result_queue = mp.Queue()
    prod = mp.Process(target=producer, args=(shm.name, size))
    cons = mp.Process(target=consumer, args=(shm.name, size, result_queue))
    prod.start()
    cons.start()
    prod.join()
    cons.join()
    if not result_queue.empty():
        time_taken, result = result_queue.get()
        print(f'Shared Memory Time: {time_taken:.3f} s, Consumer Read: {result}')
    else:
        print('Consumer did not return results.')
    shm.close()
    shm.unlink()


Consumer did not return results.


### Visualization: Shared Memory Performance
Visualize the time taken for shared memory operations.


In [ ]:
# # The following is a Chart.js JSON block for visualization. In Jupyter, use a compatible extension to render it.
# {
#   "type": "bar",
#   "data": {
#     "labels": ["Producer", "Consumer"],
#     "datasets": [{
#       "label": "Execution Time (s)",
#       "data": [0.1, 0.15],
#       "backgroundColor": ["#36A2EB", "#FF6384"],
#       "borderColor": ["#2C83C3", "#D9546E"],
#       "borderWidth": 1
#     }]
#   },
#   "options": {
#     "scales": {
#       "y": {
#         "beginAtZero": true,
#         "title": { "display": true, "text": "Time (s)" }
#       },
#       "x": {
#         "title": { "display": true, "text": "Process Role" }
#       }
#     },
#     "plugins": {
#       "legend": { "display": true },
#       "title": { "display": true, "text": "Shared Memory IPC Performance" }
#     }
#   }
# }


NameError: name 'true' is not defined

### Research Direction
- **Cache Coherence**: Optimize shared memory for multi-core systems.
- **Scalability**: Investigate shared memory in distributed systems (e.g., RDMA).


## 2. Message Passing

### Theory
**Message Passing** involves processes exchanging data via explicit messages, typically through OS-provided mechanisms (e.g., pipes, queues).

- **Mechanism**: Processes send/receive messages via a kernel or library interface.
- **Advantages**: No shared data, reducing synchronization issues.
- **Challenges**: Higher overhead due to copying and kernel involvement.

**Rare Insight**: In microkernel OS (e.g., seL4), message passing is the primary IPC mechanism, but its performance is limited by IPC latency, a key research challenge.

**Applications**:
- **Distributed Systems**: Message passing in MPI for HPC.
- **Microservices**: Message queues (e.g., RabbitMQ) for service communication.

### Practical Code: Message Passing Simulation
Simulate message passing using `multiprocessing.Queue`.


In [3]:
import multiprocessing as mp
import time

def sender(queue):
    for i in range(5):
        queue.put(f'Message-{i}')
        time.sleep(0.02)  # Simulate work

def receiver(queue, result_queue):
    start_time = time.time()
    messages = []
    while len(messages) < 5:
        if not queue.empty():
            messages.append(queue.get())
        time.sleep(0.01)
    elapsed = time.time() - start_time
    result_queue.put((elapsed, messages))

# Simulate message passing
if __name__ == '__main__':
    queue = mp.Queue()
    result_queue = mp.Queue()
    send_proc = mp.Process(target=sender, args=(queue,))
    recv_proc = mp.Process(target=receiver, args=(queue, result_queue))
    send_proc.start()
    recv_proc.start()
    send_proc.join()
    recv_proc.join()
    if not result_queue.empty():
        time_taken, messages = result_queue.get()
        print(f'Message Passing Time: {time_taken:.3f} s, Received: {messages}')
    else:
        print('Receiver did not return results.')


Receiver did not return results.


### Visualization: Message Passing Performance
Compare execution times of sender and receiver.


In [ ]:
# The following is a Chart.js JSON block for visualization. In Jupyter, use a compatible extension to render it.
{
  "type": "bar",
  "data": {
    "labels": ["Sender", "Receiver"],
    "datasets": [{
      "label": "Execution Time (s)",
      "data": [0.1, 0.12],
      "backgroundColor": ["#36A2EB", "#FF6384"],
      "borderColor": ["#2C83C3", "#D9546E"],
      "borderWidth": 1
    }]
  },
  "options": {
    "scales": {
      "y": {
        "beginAtZero": true,
        "title": { "display": true, "text": "Time (s)" }
      },
      "x": {
        "title": { "display": true, "text": "Process Role" }
      }
    },
    "plugins": {
      "legend": { "display": true },
      "title": { "display": true, "text": "Message Passing IPC Performance" }
    }
  }
}


### Research Direction
- **Low-latency IPC**: Optimize message passing in microkernels (e.g., seL4).
- **Distributed IPC**: Explore message passing in large-scale systems (e.g., MPI optimizations).


## Tutorial: Exploring IPC in Ubuntu

**Objective**: Use Ubuntu to explore IPC mechanisms with Python and system tools.

**Steps**:
1. Install VirtualBox (https://www.virtualbox.org/) and Ubuntu (https://ubuntu.com/download).
2. Create a VM (2GB RAM, 20GB storage, 2 CPUs).
3. Install Python and dependencies:
   ```bash
   sudo apt update
   sudo apt install python3 python3-pip
   pip install pandas matplotlib numpy
   ```
4. Explore IPC tools:
   ```bash
   ipcs -m  # List shared memory segments
   ipcs -q  # List message queues
   strace -e shm python3 script.py  # Trace shared memory calls
   ```

**Code**: Simulate IPC monitoring.


In [ ]:
import pandas as pd

# Mock IPC data
ipc_data = [
    {'ID': 1, 'Type': 'Shared Memory', 'Size': '4KB', 'Owner': 'user1'},
    {'ID': 2, 'Type': 'Message Queue', 'Messages': 5, 'Owner': 'user2'}
]

print(pd.DataFrame(ipc_data))


## Mini Project: Producer-Consumer with Shared Memory and Message Passing

**Objective**: Simulate a producer-consumer problem using both shared memory and message passing, comparing performance.

**Code**:


In [ ]:
import multiprocessing as mp
import numpy as np
import time
import sys

try:
    from multiprocessing import shared_memory
except ImportError:
    print('multiprocessing.shared_memory is only available in Python 3.8+')
    sys.exit(1)

def shared_memory_producer(shm_name, size, lock):
    shm = shared_memory.SharedMemory(name=shm_name)
    buffer = np.ndarray((size,), dtype=np.int32, buffer=shm.buf)
    for i in range(size):
        with lock:
            buffer[i] = i + 1  # Avoid zero
        time.sleep(0.01)
    shm.close()

def shared_memory_consumer(shm_name, size, lock, result_queue):
    shm = shared_memory.SharedMemory(name=shm_name)
    buffer = np.ndarray((size,), dtype=np.int32, buffer=shm.buf)
    start_time = time.time()
    data = []
    for i in range(size):
        while True:
            with lock:
                if buffer[i] != 0:
                    data.append(buffer[i])
                    buffer[i] = 0
                    break
            time.sleep(0.01)
    elapsed = time.time() - start_time
    result_queue.put((elapsed, data))
    shm.close()

def message_passing_producer(queue):
    for i in range(5):
        queue.put(i + 1)
        time.sleep(0.02)

def message_passing_consumer(queue, result_queue):
    start_time = time.time()
    data = []
    while len(data) < 5:
        if not queue.empty():
            data.append(queue.get())
        time.sleep(0.01)
    elapsed = time.time() - start_time
    result_queue.put((elapsed, data))

# Simulate
if __name__ == '__main__':
    # Shared Memory
    size = 5
    shm = shared_memory.SharedMemory(create=True, size=size * 4)
    buffer = np.ndarray((size,), dtype=np.int32, buffer=shm.buf)
    buffer[:] = 0
    lock = mp.Lock()
    sm_result_queue = mp.Queue()
    sm_prod = mp.Process(target=shared_memory_producer, args=(shm.name, size, lock))
    sm_cons = mp.Process(target=shared_memory_consumer, args=(shm.name, size, lock, sm_result_queue))
    sm_prod.start()
    sm_cons.start()
    sm_prod.join()
    sm_cons.join()
    if not sm_result_queue.empty():
        sm_time, sm_data = sm_result_queue.get()
        print(f'Shared Memory Time: {sm_time:.3f} s, Data: {sm_data}')
    else:
        print('Shared memory consumer did not return results.')
    shm.close()
    shm.unlink()

    # Message Passing
    queue = mp.Queue()
    mp_result_queue = mp.Queue()
    mp_prod = mp.Process(target=message_passing_producer, args=(queue,))
    mp_cons = mp.Process(target=message_passing_consumer, args=(queue, mp_result_queue))
    mp_prod.start()
    mp_cons.start()
    mp_prod.join()
    mp_cons.join()
    if not mp_result_queue.empty():
        mp_time, mp_data = mp_result_queue.get()
        print(f'Message Passing Time: {mp_time:.3f} s, Data: {mp_data}')
    else:
        print('Message passing consumer did not return results.')


## Major Project: IPC Framework

**Objective**: Design an IPC framework supporting shared memory and message passing with performance analysis.

**Outline**:
- **Framework**: Supports both IPC mechanisms with synchronization.
- **Performance Metrics**: Measure latency and throughput.
- **Extensibility**: Allow custom message formats or memory sizes.

**Code**:


In [ ]:
import multiprocessing as mp
import numpy as np
import time
import pandas as pd
import sys

try:
    from multiprocessing import shared_memory
except ImportError:
    print('multiprocessing.shared_memory is only available in Python 3.8+')
    sys.exit(1)

class IPCFramework:
    def __init__(self):
        self.shm = None
        self.queue = mp.Queue()
        self.lock = mp.Lock()
        self.metrics = []

    def init_shared_memory(self, size):
        self.shm = shared_memory.SharedMemory(create=True, size=size * 4)
        buffer = np.ndarray((size,), dtype=np.int32, buffer=self.shm.buf)
        buffer[:] = 0

    def shared_memory_write(self, data):
        start_time = time.time()
        buffer = np.ndarray((len(data),), dtype=np.int32, buffer=self.shm.buf)
        with self.lock:
            buffer[:len(data)] = data
        self.metrics.append(('Shared Memory Write', (time.time() - start_time) * 1000))

    def shared_memory_read(self, size):
        start_time = time.time()
        buffer = np.ndarray((size,), dtype=np.int32, buffer=self.shm.buf)
        with self.lock:
            data = buffer[:size].tolist()
        self.metrics.append(('Shared Memory Read', (time.time() - start_time) * 1000))
        return data

    def message_passing_send(self, data):
        start_time = time.time()
        for item in data:
            self.queue.put(item)
        self.metrics.append(('Message Passing Send', (time.time() - start_time) * 1000))

    def message_passing_receive(self, count):
        start_time = time.time()
        data = []
        for _ in range(count):
            data.append(self.queue.get())
        self.metrics.append(('Message Passing Receive', (time.time() - start_time) * 1000))
        return data

    def get_metrics(self):
        return pd.DataFrame(self.metrics, columns=['Operation', 'Time (ms)'])

    def cleanup(self):
        if self.shm:
            self.shm.close()
            self.shm.unlink()

# Simulate
if __name__ == '__main__':
    ipc = IPCFramework()
    ipc.init_shared_memory(size=5)
    ipc.shared_memory_write([1, 2, 3, 4, 5])
    sm_data = ipc.shared_memory_read(5)
    ipc.message_passing_send([1, 2, 3, 4, 5])
    mp_data = ipc.message_passing_receive(5)
    print(f'Shared Memory Data: {sm_data}')
    print(f'Message Passing Data: {mp_data}')
    print(ipc.get_metrics())
    ipc.cleanup()


## Additional Content for Scientists

### Security Considerations
- **Shared Memory**: Protect memory segments from unauthorized access (e.g., memory isolation).
- **Message Passing**: Ensure secure message channels to prevent interception.

### Performance Optimization
- **Shared Memory**: Use cache-aligned data structures to reduce coherence overhead.
- **Message Passing**: Optimize queue implementations for low latency.

### Advanced Formal Modeling
- **Process Algebras**: Use CSP to model deadlock-free IPC.
- **State Machines**: Model shared memory access with synchronization.

**Code**: Conceptual CSP model for IPC.


In [ ]:
class CSPModel:
    def __init__(self):
        self.events = []

    def ipc_event(self, process, event):
        self.events.append(f'{process} -> {event}')

# Simulate
csp = CSPModel()
csp.ipc_event('P1', 'send_message')
csp.ipc_event('P2', 'receive_message')
print(csp.events)


## Conclusion

This notebook provides a comprehensive exploration of IPC, covering shared memory and message passing. Key takeaways:
- Shared memory offers high performance but requires synchronization.
- Message passing is safer but slower due to copying overhead.
- Practical simulations and tools enable hands-on learning.

**Next Steps**:
- Extend the IPC framework to support pipes or sockets.
- Implement real IPC in C (e.g., using POSIX shm_open).
- Use formal verification tools (e.g., Isabelle/HOL) to prove IPC correctness.
